# Enviorment

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from  matplotlib import pyplot as plt
import networkx as nx
plt.style.use('ggplot')
from tqdm import tqdm_notebook
#tqdm_notebook().pandas()
import swifter
import qgrid
import numpy as np
import networkx as nx
import random
from scipy.sparse import spdiags
from functools import partial
import mygene
import sys

# Loading STRING network

## STRING(addressed)

In [2]:
file = '../data/string.txt'
df_og_string = pd.read_table(file,sep=' ',names=['protein1','protein2','combined_score'])
print(df_og_string)
df_og = np.append(df_og_string.protein1.unique(),df_og_string.protein2.unique())
print(len(df_og))

                 protein1         protein2  combined_score
0         ENSP00000000233  ENSP00000272298             490
1         ENSP00000000233  ENSP00000253401             198
2         ENSP00000000233  ENSP00000401445             159
3         ENSP00000000233  ENSP00000418915             606
4         ENSP00000000233  ENSP00000327801             167
...                   ...              ...             ...
11759449  ENSP00000485678  ENSP00000310488             167
11759450  ENSP00000485678  ENSP00000342448             175
11759451  ENSP00000485678  ENSP00000350222             195
11759452  ENSP00000485678  ENSP00000367590             900
11759453  ENSP00000485678  ENSP00000349930             213

[11759454 rows x 3 columns]
38708


## STRING-physical(unaddressed)

In [3]:
file='../data/9606.protein.physical.links.detailed.v11.0.txt'
df_string = pd.read_csv(file,sep=' ')
print(df_string)

                     protein1              protein2  experiments  database  \
0        9606.ENSP00000000233  9606.ENSP00000003100           85         0   
1        9606.ENSP00000000233  9606.ENSP00000005260          262         0   
2        9606.ENSP00000000233  9606.ENSP00000007414           83         0   
3        9606.ENSP00000000233  9606.ENSP00000009105          167         0   
4        9606.ENSP00000000233  9606.ENSP00000012443          157         0   
...                       ...                   ...          ...       ...   
8683833  9606.ENSP00000485672  9606.ENSP00000469863          178         0   
8683834  9606.ENSP00000485672  9606.ENSP00000479667          266         0   
8683835  9606.ENSP00000485678  9606.ENSP00000248572            0       900   
8683836  9606.ENSP00000485678  9606.ENSP00000334051            0       900   
8683837  9606.ENSP00000485678  9606.ENSP00000481878            0       900   

         textmining  combined_score  
0                65      

### string address

In [4]:
df_string['protein1'] = df_string.protein1.apply(lambda x : x.lstrip('9606.'))
df_string['protein2'] = df_string.protein2.apply(lambda x : x.lstrip('9606.'))
df_string[:3]

,protein1,protein2,experiments,database,textmining,combined_score
0,ENSP00000000233,ENSP00000003100,85,0,65,107
1,ENSP00000000233,ENSP00000005260,262,0,0,262
2,ENSP00000000233,ENSP00000007414,83,0,108,147


### value address

In [5]:
new_col = 'percent'
def col_percent(df):
    return (df['combined_score']-df['textmining'])/1000

In [6]:
df_string[new_col] = df_string.swifter.apply(col_percent,axis=1)
confidence_score_high = df_string[df_string.percent >= 0.7]
confidence_score_med = df_string[(df_string.percent<0.7) & (df_string.percent >=0.4)]

In [7]:
df_string[:3]

,protein1,protein2,experiments,database,textmining,combined_score,percent
0,ENSP00000000233,ENSP00000003100,85,0,65,107,0.042
1,ENSP00000000233,ENSP00000005260,262,0,0,262,0.262
2,ENSP00000000233,ENSP00000007414,83,0,108,147,0.039


In [8]:
print(confidence_score_high.describe())

         experiments       database     textmining  combined_score  \
count  221372.000000  221372.000000  221372.000000   221372.000000   
mean      124.226524     862.972734      45.396057      900.830484   
std       231.571236     161.380010      58.996919       42.885821   
min         0.000000       0.000000       0.000000      700.000000   
25%         0.000000     900.000000       0.000000      900.000000   
50%         0.000000     900.000000       0.000000      901.000000   
75%       162.000000     900.000000      69.000000      912.000000   
max       998.000000     900.000000     298.000000      999.000000   

             percent  
count  221372.000000  
mean        0.855434  
std         0.059726  
min         0.700000  
25%         0.824000  
50%         0.867000  
75%         0.900000  
max         0.999000  


# nodelist

In [9]:
physical_nodelist = np.append(df_string.protein1.unique(),df_string.protein2.unique())

In [10]:
physical_nodelist =list(set(physical_nodelist))
print(len(physical_nodelist))
print(physical_nodelist[:5])

19000
['ENSP00000376423', 'ENSP00000322915', 'ENSP00000380178', 'ENSP00000344401', 'ENSP00000248706']


## read STRING-phycial addressed graph to graph

txt to graph

In [11]:
'''
import networkx as nx
fh = open(r"/Users/suweilin/Downloads/STRING_physical_network_AfterProcess.txt", 'rb')
G1 = nx.read_weighted_edgelist(fh,create_using=nx.DiGraph)
degree1 = nx.degree(G1, nbunch=None, weight=True)
numberofnode1 = nx.number_of_nodes(G1)
numberofedge1 = nx.number_of_edges(G1)
node1 =list(nx.nodes(G1))
print(len(node1))
edge1 = nx.edges(G1)
print(nx.info(G1))
fh.close()
'''

'\nimport networkx as nx\nfh = open(r"/Users/suweilin/Downloads/STRING_physical_network_AfterProcess.txt", \'rb\')\nG1 = nx.read_weighted_edgelist(fh,create_using=nx.DiGraph)\ndegree1 = nx.degree(G1, nbunch=None, weight=True)\nnumberofnode1 = nx.number_of_nodes(G1)\nnumberofedge1 = nx.number_of_edges(G1)\nnode1 =list(nx.nodes(G1))\nprint(len(node1))\nedge1 = nx.edges(G1)\nprint(nx.info(G1))\nfh.close()\n'

pandas dataframe to graph

In [12]:
Graph_string_physical = nx.from_pandas_edgelist(df_string,source='protein1',target='protein2',create_using=nx.DiGraph())
print(nx.info(Graph_string_physical))

Name: 
Type: DiGraph
Number of nodes: 19000
Number of edges: 8683838
Average in degree: 457.0441
Average out degree: 457.0441


high confidence selected

In [13]:
Graph_string_physical_highconfidence = nx.from_pandas_edgelist(confidence_score_high,source = 'protein1', target = 'protein2', edge_attr=['percent'],create_using=nx.DiGraph())
print(nx.info(Graph_string_physical_highconfidence))
confidence_score_high_nodelist = list(nx.nodes(Graph_string_physical_highconfidence))
print(len(confidence_score_high_nodelist))

Name: 
Type: DiGraph
Number of nodes: 7845
Number of edges: 221372
Average in degree:  28.2182
Average out degree:  28.2182
7845


# transfer ID to different databas

## STRING phycisal 

Ensembl protein to ensembl gene

In [14]:
mg = mygene.MyGeneInfo()
results_string_physical= mg.querymany(physical_nodelist, scopes='ensembl.protein',fields = 'ensembl.gene',species='human',returnall=True)
#19000個輸入，然後有608個找不到transfer gene id，但是這邊的數字，怎麼會比high confidence score的部分還要少...

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
Finished.
608 input query terms found no hit:
	['ENSP00000327585', 'ENSP00000332110', 'ENSP00000364964', 'ENSP00000321246', 'ENSP00000062104', 'ENS


Ensembl protein to KEGG

In [15]:
results_string_physical_kegg = mg.querymany(physical_nodelist, scopes='ensembl.protein',fields = 'pathway.kegg',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
print(results_string_physical_kegg)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
Finished.
608 input query terms found no hit:
	['ENSP00000327585', 'ENSP00000332110', 'ENSP00000364964', 'ENSP00000321246', 'ENSP00000062104', 'ENS
{'out':                     _id     _score  \
query                                
ENSP00000376423    6700  22.414150   
ENSP00000322915    9202  23.146717   
ENSP00000380178    7326  22.414150   
ENSP00000344401  389421  21.904642   
ENSP00000248706   65997  22.414150   
...                 ...        ...   
ENSP0000034

In [16]:
results_string_physical_kegg_id = mg.querymany(physical_nodelist, scopes='ensembl.protein',fields = 'pathway.kegg.id',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
print(results_string_physical_kegg_id)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
Finished.
608 input query terms found no hit:
	['ENSP00000327585', 'ENSP00000332110', 'ENSP00000364964', 'ENSP00000321246', 'ENSP00000062104', 'ENS
{'out':                     _id     _score  \
query                                
ENSP00000376423    6700  22.414150   
ENSP00000322915    9202  23.146717   
ENSP00000380178    7326  22.414150   
ENSP00000344401  389421  21.904642   
ENSP00000248706   65997  22.414150   
...                 ...        ...   
ENSP0000034

Ensembl protein to GO

In [17]:
results_string_physical_go = mg.querymany(physical_nodelist, scopes='ensembl.protein',fields = 'go',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
print(results_string_physical_go)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
Finished.
608 input query terms found no hit:
	['ENSP00000327585', 'ENSP00000332110', 'ENSP00000364964', 'ENSP00000321246', 'ENSP00000062104', 'ENS
{'out':                     _id     _score  \
query                                
ENSP00000376423    6700  22.419800   
ENSP00000322915    9202  22.425173   
ENSP00000380178    7326  22.419800   
ENSP00000344401  389421  21.904642   
ENSP00000248706   65997  22.419800   
...                 ...        ...   
ENSP0000034

Ensembl protein to Reactome

In [18]:
results_string_physical_reactome = mg.querymany(physical_nodelist, scopes='ensembl.protein',fields = 'pathway.reactome.id',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
print(results_string_physical_reactome)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
Finished.
608 input query terms found no hit:
	['ENSP00000327585', 'ENSP00000332110', 'ENSP00000364964', 'ENSP00000321246', 'ENSP00000062104', 'ENS
{'out':                     _id     _score  \
query                                
ENSP00000376423    6700  22.419800   
ENSP00000322915    9202  23.146717   
ENSP00000380178    7326  22.419800   
ENSP00000344401  389421  21.469257   
ENSP00000248706   65997  22.419800   
...                 ...        ...   
ENSP0000034

## STRING phycial high confidence

ensembl protein to ensembl gene

In [ ]:
#confidence_score_high_nodelist = set(np.append(confidence_score_high.protein1.unique(),confidence_score_high.protein2.unique()))
confidence_score_high_nodelist = list(nx.nodes(Graph_string_physical_highconfidence))
mg = mygene.MyGeneInfo()
results_high= mg.querymany(confidence_score_high_nodelist, scopes='ensembl.protein',fields = 'ensembl.gene',species='human',returnall=True,as_dataframe=True)
print(results_high)

Ensembl protein to KEGG

In [ ]:
results_high_kegg= mg.querymany(confidence_score_high_nodelist, scopes='ensembl.protein',fields = 'pathway.kegg',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
print(results_high_kegg)

Ensembel protein to GO

In [ ]:
results_high_go= mg.querymany(confidence_score_high_nodelist, scopes='ensembl.protein',fields = 'go',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
print(results_high_go)

Ensembl protein to Reactome

In [ ]:
results_high_reactome= mg.querymany(confidence_score_high_nodelist, scopes='ensembl.protein',fields = 'reactome',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
print(results_high_rectome)

# extract KEGG id

## STRING physical network with kegg

In [22]:
print(results_string_physical_kegg)

{'out':                     _id     _score  \
query                                
ENSP00000376423    6700  22.414150   
ENSP00000322915    9202  23.146717   
ENSP00000380178    7326  22.414150   
ENSP00000344401  389421  21.904642   
ENSP00000248706   65997  22.414150   
...                 ...        ...   
ENSP00000349575  126308  21.833403   
ENSP00000364691    9696  21.833403   
ENSP00000276708   56169  21.515104   
ENSP00000448998   66002  21.515104   
ENSP00000386988  387707  21.833403   

                                                      pathway.kegg  \
query                                                                
ENSP00000376423                                                NaN   
ENSP00000322915                                                NaN   
ENSP00000380178  [{'id': 'hsa04120', 'name': 'Ubiquitin mediate...   
ENSP00000344401                                                NaN   
ENSP00000248706                                                NaN   
...    

In [23]:
print(results_string_physical_kegg['out']['pathway.kegg.id'])

query
ENSP00000376423    NaN
ENSP00000322915    NaN
ENSP00000380178    NaN
ENSP00000344401    NaN
ENSP00000248706    NaN
                  ... 
ENSP00000349575    NaN
ENSP00000364691    NaN
ENSP00000276708    NaN
ENSP00000448998    NaN
ENSP00000386988    NaN
Name: pathway.kegg.id, Length: 19000, dtype: object


In [24]:
print(type(results_string_physical_kegg['out']))
print(type(results_string_physical_kegg['out']['pathway.kegg.id']))
#inbiomap_kegg=results_inbiomap_kegg['out']['pathway.kegg.id'].to_frame()
string_physical_kegg = results_string_physical_kegg['out'].reset_index()
#inbiomap_kegg = inbiomap_kegg.reset_index()
#這邊會需要這一個method是因為丟入的geneid 會變成index，會因此不好取用裡面的值，所以使用reset index來重置
#print(type(inbiomap_kegg))
string_physical_kegg = string_physical_kegg.rename(mapper={'pathway.kegg':'pathway_kegg','pathway.kegg.id':'pathway_kegg_id','pathway.kegg.name':'pathway_kegg_name'},axis=1)
#做rename，是為了後面比較好分析
print(string_physical_kegg.info())
print(string_phycial_kegg.describe())
string_phycial_kegg[:5]

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19000 entries, 0 to 18999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   query              19000 non-null  object 
 1   _id                18392 non-null  object 
 2   _score             18392 non-null  float64
 3   pathway_kegg       4035 non-null   object 
 4   pathway_kegg_id    3450 non-null   object 
 5   pathway_kegg_name  3450 non-null   object 
 6   notfound           608 non-null    object 
dtypes: float64(1), object(6)
memory usage: 1.0+ MB
None


NameError: name 'string_phycial_kegg' is not defined

check kegg id

In [25]:
#print(inbiomap_kegg['pathway.kegg.id'])
string_physical_kegg_id = string_physical_kegg['pathway_kegg_id'].unique()
print(len(string_physical_kegg_id))
#print(inbiomap_kegg_id)

274


 extract needed dataframe

In [26]:
string_physical_kegg_pathwaylist = string_physical_kegg[['query','pathway_kegg_id']]
print(string_physical_kegg_pathwaylist)

                 query pathway_kegg_id
0      ENSP00000376423             NaN
1      ENSP00000322915             NaN
2      ENSP00000380178             NaN
3      ENSP00000344401             NaN
4      ENSP00000248706             NaN
...                ...             ...
18995  ENSP00000349575             NaN
18996  ENSP00000364691             NaN
18997  ENSP00000276708             NaN
18998  ENSP00000448998             NaN
18999  ENSP00000386988             NaN

[19000 rows x 2 columns]


address nan

In [27]:
string_physical_kegg_pathwaylist['pathway_kegg_id'] = string_physical_kegg_pathwaylist['pathway_kegg_id'].map(np.nan_to_num)
print(string_physical_kegg_pathwaylist[:3])

             query pathway_kegg_id
0  ENSP00000376423               0
1  ENSP00000322915               0
2  ENSP00000380178               0


<ipython-input-27-8da98129adb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  string_physical_kegg_pathwaylist['pathway_kegg_id'] = string_physical_kegg_pathwaylist['pathway_kegg_id'].map(np.nan_to_num)


eliminate nan

In [28]:
i = string_physical_kegg_pathwaylist[string_physical_kegg_pathwaylist.pathway_kegg_id == 0 ].index
print(len(i))
string_physical_kegg_pathwaylist = string_physical_kegg_pathwaylist.drop(i)

15550


### construct heterogeneous network

pd_dataframe_inbiomap_and_kegg

In [29]:
string_physical_kegg_pathwaylist = string_physical_kegg_pathwaylist.rename(mapper={'query':'protein1','pathway_kegg_id':'protein2'},axis=1)
string_physical_kegg_pathwaylist

,protein1,protein2
8,ENSP00000297071,hsa03040
12,ENSP00000220966,hsa00330
19,ENSP00000340736,hsa00500
22,ENSP00000385470,hsa04060
27,ENSP00000230085,hsa04144
...,...,...
18972,ENSP00000364543,hsa03018
18974,ENSP00000377001,hsa04340
18979,ENSP00000334452,hsa04740
18988,ENSP00000365256,hsa04613


combine inbiomap df and kegg id df

In [30]:
heterogeneous_network_string_physical_kegg_df = pd.concat([df_string,string_physical_kegg_pathwaylist],axis=0)

construct heterogeneous network

In [31]:
heterogeneous_network_string_physical_kegg = nx.from_pandas_edgelist(heterogeneous_network_string_physical_kegg_df,source='protein1',target ='protein2')
print(nx.info(heterogeneous_network_string_physical_kegg))

Name: 
Type: Graph
Number of nodes: 19273
Number of edges: 4345369
Average degree: 450.9281


### write heterogeneous network as file

In [32]:
nx.write_edgelist(heterogeneous_network_string_physical_kegg, "../data/heterogeneous_STRINGPhysical_KEGG.txt")

## STRING-physical  high confidence with KEGG

In [ ]:
print(type(results_high_kegg['out']))
print(type(results_high_kegg['out']['pathway.kegg.id']))
#inbiomap_kegg=results_inbiomap_kegg['out']['pathway.kegg.id'].to_frame()
STRING_physical_high_kegg = results_high_kegg['out'].reset_index()
#inbiomap_kegg = inbiomap_kegg.reset_index()
#這邊會需要這一個method是因為丟入的geneid 會變成index，會因此不好取用裡面的值，所以使用reset index來重置
#print(type(inbiomap_kegg))
STRING_physical_high_kegg = STRING_physical_high_kegg.rename(mapper={'pathway.kegg':'pathway_kegg','pathway.kegg.id':'pathway_kegg_id','pathway.kegg.name':'pathway_kegg_name'},axis=1)
#做rename，是為了後面比較好分析
print(STRING_physical_high_kegg.info())
print(STRING_physical_high_kegg.describe())
STRING_physical_high_kegg[:5]

In [ ]:
#print(inbiomap_kegg['pathway.kegg.id'])
STRING_physical_high_kegg_id = STRING_physical_high_kegg['pathway_kegg_id'].unique()
print('有多少pathway被mapping在網路中: ',len(STRING_physical_high_kegg_id))
#print(inbiomap_kegg_id)

In [ ]:
STRING_physical_high_kegg_pathwaylist = STRING_physical_high_kegg[['query','pathway_kegg_id']]
print(STRING_physical_high_kegg_pathwaylist)

In [ ]:
STRING_physical_high_kegg_pathwaylist['pathway_kegg_id'] = STRING_physical_high_kegg_pathwaylist['pathway_kegg_id'].map(np.nan_to_num)

In [ ]:
STRING_physical_high_kegg_pathwaylist

In [ ]:
j = STRING_physical_high_kegg_pathwaylist[STRING_physical_high_kegg_pathwaylist.pathway_kegg_id == 0 ].index
print('有多少數量的gene是沒有被找到相應的pathway:',len(j))
STRING_physical_high_kegg_pathwaylist = STRING_physical_high_kegg_pathwaylist.drop(j)

In [ ]:
STRING_physical_high_kegg_pathwaylist = STRING_physical_high_kegg_pathwaylist.rename(mapper={'query':'protein1','pathway_kegg_id':'protein2'},axis=1)
STRING_physical_high_kegg_pathwaylist

In [ ]:
heterogeneous_network_STRING_physical_high_kegg_df = pd.concat([confidence_score_high,STRING_physical_high_kegg_pathwaylist],axis=0)
heterogeneous_network_STRING_physical_high_kegg_df

In [ ]:
heterogeneous_network_STRING_physical_high_kegg = nx.from_pandas_edgelist(heterogeneous_network_STRING_physical_high_kegg_df,source='protein1',target ='protein2')
print(nx.info(heterogeneous_network_STRING_physical_high_kegg))